In [4]:
import sqlite3

import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

from autogluon.tabular import TabularDataset, TabularPredictor

In [5]:
# Global variables
DB_PATH = "Data/dummy_data.db"

# Load dataset with igef (vehicle), test name, and test_result (OK/NOK)
conn = sqlite3.connect(DB_PATH)
df = pd.read_sql("SELECT * FROM dummy_data", conn)
conn.close()

In [6]:
# Define target test and split ratio
label = "test_80"
test_size = 0.2

# Convert test_result to binary (OK=0, NOK=1)
df["binary_test_result"] = df["test_result"].map({"OK": 0, "NOK": 1})

# Create target labels: one result per vehicle for test_80 (NOK overrides OK)
y = (
    df[df["test"] == label]
    .groupby("igef")["binary_test_result"]
    .max()
    .rename(label)
    .to_frame()
)

# Create feature matrix: one row per vehicle, one column per test (excluding test_80)
X = (
    df[df["test"] != label]
    .pivot_table(index="igef", columns="test", values="binary_test_result", aggfunc="max")
    .sort_index(axis=1)
    .fillna(0)
)

# Merge features with target labels
data = X.join(y, how="inner").reset_index()

# Split by vehicle, stratified to preserve OK/NOK ratio and avoid leakage
igefs = data["igef"].unique()
igef_train, igef_test = train_test_split(
    igefs,
    test_size=test_size,
    stratify=data[label],
    random_state=42
)

train = data[data["igef"].isin(igef_train)]
test = data[data["igef"].isin(igef_test)]


In [7]:
# Train AutoGluon TabularPredictor
# label: column to predict ("test_80")
# eval_metric: F1 (robust to class imbalance)
# train_data: drop vehicle ID ("igef")
# presets: balanced speed/quality config
# num_bag_folds: bagging via cross-validation for generalization
# num_stack_levels: model stacking for extra accuracy
# time_limit: max 1800 s (~30 min)
predictor = (
    TabularPredictor(label=label, eval_metric="f1")
    .fit(
        train_data=train.drop(columns=["igef"]),
        presets="medium_quality_faster_train",
        num_bag_folds=5,
        num_stack_levels=1,
        time_limit=1800
    )
)

No path specified. Models will be saved in: "AutogluonModels/ag-20251030_104950"
Preset alias specified: 'medium_quality_faster_train' maps to 'medium_quality'.
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.12.12
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 24.6.0: Mon Jul 14 11:30:34 PDT 2025; root:xnu-11417.140.69~1/RELEASE_ARM64_T8103
CPU Count:          8
Memory Avail:       5.38 GB / 16.00 GB (33.6%)
Disk Space Avail:   65.11 GB / 228.27 GB (28.5%)
Presets specified: ['medium_quality_faster_train']
Using hyperparameters preset: hyperparameters='default'
Beginning AutoGluon training ... Time limit = 1800s
AutoGluon will save models to "/Users/lpossner/Projects/databot/AutogluonModels/ag-20251030_104950"
Train Data Rows:    800
Train Data Columns: 80
Label Column:       test_80
AutoGluon infers your prediction problem is: 'binary' (because only two 

In [8]:
# Evaluate model on test set

# Show leaderboard with model performance and training stats
leaderboard = predictor.leaderboard(test.drop(columns=["igef"]), silent=True)
print(leaderboard)

# Predict test labels (drop igef)
y_true = test[label]
y_pred = predictor.predict(test.drop(columns=["igef"]))

# Print precision, recall, and F1-score
print(classification_report(y_true, y_pred))

# Show confusion matrix (true vs. predicted)
print(confusion_matrix(y_true, y_pred))

# Display top feature importances
features_importances = predictor.feature_importance(test.drop(columns=["igef"]))
print(features_importances.head(20))

Computing feature importance via permutation shuffling for 80 features using 200 rows with 5 shuffle sets...
	6.79s	= Expected runtime (1.36s per shuffle set)


                      model  score_test  score_val eval_metric  \
0           CatBoost_BAG_L1    1.000000   1.000000          f1   
1       WeightedEnsemble_L2    1.000000   1.000000          f1   
2    NeuralNetFastAI_BAG_L1    1.000000   1.000000          f1   
3      LightGBMLarge_BAG_L2    1.000000   1.000000          f1   
4           LightGBM_BAG_L2    1.000000   1.000000          f1   
5           CatBoost_BAG_L2    1.000000   1.000000          f1   
6            XGBoost_BAG_L2    1.000000   0.987952          f1   
7     NeuralNetTorch_BAG_L2    1.000000   1.000000          f1   
8    NeuralNetFastAI_BAG_L2    1.000000   1.000000          f1   
9     ExtraTreesGini_BAG_L2    1.000000   1.000000          f1   
10      WeightedEnsemble_L3    1.000000   1.000000          f1   
11  RandomForestEntr_BAG_L2    1.000000   1.000000          f1   
12  RandomForestGini_BAG_L2    1.000000   1.000000          f1   
13    ExtraTreesEntr_BAG_L2    1.000000   1.000000          f1   
14     Lig

	0.57s	= Actual runtime (Completed 5 of 5 shuffle sets)


         importance    stddev   p_value  n  p99_high   p99_low
test_2     0.360000  0.036515  0.000013  5  0.435185  0.284815
test_1     0.283072  0.091498  0.001146  5  0.471467  0.094677
test_4     0.214436  0.035653  0.000088  5  0.287846  0.141026
test_3     0.098195  0.045124  0.004122  5  0.191107  0.005284
test_61    0.000000  0.000000  0.500000  5  0.000000  0.000000
test_60    0.000000  0.000000  0.500000  5  0.000000  0.000000
test_6     0.000000  0.000000  0.500000  5  0.000000  0.000000
test_59    0.000000  0.000000  0.500000  5  0.000000  0.000000
test_58    0.000000  0.000000  0.500000  5  0.000000  0.000000
test_57    0.000000  0.000000  0.500000  5  0.000000  0.000000
test_56    0.000000  0.000000  0.500000  5  0.000000  0.000000
test_0     0.000000  0.000000  0.500000  5  0.000000  0.000000
test_55    0.000000  0.000000  0.500000  5  0.000000  0.000000
test_62    0.000000  0.000000  0.500000  5  0.000000  0.000000
test_53    0.000000  0.000000  0.500000  5  0.000000  0